In [72]:
import yfinance as yf
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

# Step 1: Download Data from yFinance
data = yf.download('GBPUSD=X', start='2000-01-01', end='2023-01-01', interval='1d')
data = data['Adj Close']  # Use the adjusted close price

# Step 2: Resample to Wednesday-to-Wednesday weekly data
weekly_data = data.resample('W-WED').last()
log_returns = np.log(weekly_data / weekly_data.shift(1)).dropna()

# Convert to PyTorch tensors
returns = torch.tensor(log_returns.values, dtype=torch.float32).view(-1, 1)

# Step 3: Create Dataset and DataLoader
class ReturnsDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        return (
            self.data[idx:idx + self.seq_len],  # Input sequence
            self.data[idx + self.seq_len]      # Target value
        )

seq_len = 1
dataset = ReturnsDataset(returns, seq_len)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Step 4: Define LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc_mu = nn.Linear(hidden_size, 1)
        self.fc_sigma = nn.Linear(hidden_size, 1)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)  # LSTM output
        hn = hn[-1]  # Take the last hidden state
        mu = self.fc_mu(hn)
        sigma = torch.exp(self.fc_sigma(hn))  # Ensure positive stddev
        return mu, sigma

# Model, Loss, and Optimizer
model = LSTMModel(input_size=1, hidden_size=50)
criterion = lambda mu, sigma, y: torch.mean(0.5 * torch.log(2 * np.pi * sigma ** 2) + (y - mu) ** 2 / (2 * sigma ** 2))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Step 5: Train the Model
n_epochs = 20
model.train()
for epoch in range(n_epochs):
    epoch_loss = 0
    for inputs, targets in dataloader:
        inputs = inputs  # Add feature dimension
        targets = targets

        optimizer.zero_grad()
        mu, sigma = model(inputs)
        loss = criterion(mu, sigma, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {epoch_loss / len(dataloader):.4f}")

# Step 6: Visualize Results Using Bokeh
model.eval()
preds_mu = []
preds_sigma = []
actual = []
with torch.no_grad():
    for i in range(len(returns) - seq_len):
        seq = returns[i:i + seq_len]
        mu, sigma = model(seq)
        preds_mu.append(mu.item())
        preds_sigma.append(sigma.item())
        actual.append(returns[i + seq_len].item())

# Convert to Pandas DataFrame
predictions = pd.DataFrame({
    'Actual': actual,
    'Predicted_Mean': preds_mu,
    'Predicted_Std': preds_sigma
}, index=log_returns.index[seq_len:])

# Calculate 5th and 95th percentiles
predictions['Lower_Band'] = predictions['Predicted_Mean'] - 1.645 * predictions['Predicted_Std']
predictions['Upper_Band'] = predictions['Predicted_Mean'] + 1.645 * predictions['Predicted_Std']

# Identify exceedances
predictions['Exceeds_Upper'] = (predictions['Actual'] > predictions['Upper_Band'])
predictions['Exceeds_Lower'] = (predictions['Actual'] < predictions['Lower_Band'])
exceedances_upper_count = predictions['Exceeds_Upper'].sum()
exceedances_lower_count = predictions['Exceeds_Lower'].sum()

# Bokeh Visualization
output_notebook()
source = ColumnDataSource(predictions)
p = figure(title="GBP/USD Weekly Log Returns: Actual vs. Predicted", x_axis_type="datetime", width=800, height=400)

# Plot actual and predicted mean
p.line('Date', 'Actual', source=source, color='blue', legend_label='Actual', line_width=2)
p.line('Date', 'Predicted_Mean', source=source, color='green', legend_label='Predicted Mean', line_width=2)

# Plot uncertainty band
p.varea(x='Date', y1='Lower_Band', y2='Upper_Band', source=source, color='green', alpha=0.2, legend_label='Uncertainty Band')

# Highlight exceedances
exceeds_upper_source = ColumnDataSource(predictions[predictions['Exceeds_Upper']])
p.circle('Date', 'Actual', source=exceeds_upper_source, color='red', size=6, legend_label='Upper Exceedances')
exceeds_lower_source = ColumnDataSource(predictions[predictions['Exceeds_Lower']])
p.circle('Date', 'Actual', source=exceeds_lower_source, color='orange', size=6, legend_label='Lower Exceedances')


# Add legend and labels
#p.legend.location = "top_left"
p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "Log Return"

# Display exceedance count
print(f"Number of upper exceedances: {exceedances_upper_count}")
print(f"Number of lower exceedances: {exceedances_lower_count}")

show(p)


[*********************100%***********************]  1 of 1 completed


Epoch 1/20, Loss: 0.7848
Epoch 2/20, Loss: 0.6360
Epoch 3/20, Loss: 0.4199
Epoch 4/20, Loss: 0.0632
Epoch 5/20, Loss: -0.4994
Epoch 6/20, Loss: -1.3125
Epoch 7/20, Loss: -2.2849
Epoch 8/20, Loss: -2.8104
Epoch 9/20, Loss: -2.7968
Epoch 10/20, Loss: -2.8413
Epoch 11/20, Loss: -2.8664
Epoch 12/20, Loss: -2.8743
Epoch 13/20, Loss: -2.8367
Epoch 14/20, Loss: -2.8610
Epoch 15/20, Loss: -2.7825
Epoch 16/20, Loss: -2.8416
Epoch 17/20, Loss: -2.7577
Epoch 18/20, Loss: -2.8112
Epoch 19/20, Loss: -2.8479
Epoch 20/20, Loss: -2.8519


Loading BokehJS ...

Number of upper exceedances: 39
Number of lower exceedances: 32


In [52]:
import yfinance as yf
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

# Step 1: Download Data from yFinance
data_gbp = yf.download('GBPUSD=X', start='2000-01-01', end='2023-01-01', interval='1d')
data_eur = yf.download('EURUSD=X', start='2000-01-01', end='2023-01-01', interval='1d')
data_gbp = data_gbp['Adj Close']  # Use the adjusted close price
data_eur = data_eur['Adj Close']

# Step 2: Resample to Wednesday-to-Wednesday weekly data
weekly_gbp = data_gbp.resample('W-WED').last()
weekly_eur = data_eur.resample('W-WED').last()
log_returns_gbp = np.log(weekly_gbp / weekly_gbp.shift(1)).dropna()
log_returns_eur = np.log(weekly_eur / weekly_eur.shift(1)).dropna()

# Combine returns into a single DataFrame
log_returns = pd.DataFrame({
    'GBPUSD': log_returns_gbp,
    'EURUSD': log_returns_eur
}).dropna()

# Convert to PyTorch tensors
returns = torch.tensor(log_returns.values, dtype=torch.float32)

# Step 3: Create Dataset and DataLoader
class ReturnsDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        return (
            self.data[idx:idx + self.seq_len],  # Input sequence
            self.data[idx + self.seq_len]      # Target values (both GBP and EUR)
        )

seq_len = 10
dataset = ReturnsDataset(returns, seq_len)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Step 4: Define LSTM Model with Correlated Normals
class CorrelatedLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_hidden_layer):
        super(CorrelatedLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_hidden_layer, batch_first=True)
        self.fc_mu = nn.Linear(hidden_size, 2)  # Mean for GBP/USD and EUR/USD
        self.fc_sigma = nn.Linear(hidden_size, 2)  # Standard deviations for GBP/USD and EUR/USD
        self.fc_rho = nn.Linear(hidden_size, 1)  # Correlation coefficient

    def forward(self, x):
        _, (hn, _) = self.lstm(x)  # LSTM output
        hn = hn[-1]  # Take the last hidden state
        mu = self.fc_mu(hn)
        sigma = torch.exp(self.fc_sigma(hn))  # Ensure positive stddev
        rho = torch.tanh(self.fc_rho(hn))  # Correlation coefficient between -1 and 1
        return mu, sigma, rho

# Model, Loss, and Optimizer
model = CorrelatedLSTMModel(input_size=2, hidden_size=50, num_hidden_layer =3)

# Define custom loss function for correlated normals
def correlated_normal_loss(mu, sigma, rho, y):
    diff = y - mu
    z1 = diff[:, 0] / sigma[:, 0]
    z2 = diff[:, 1] / sigma[:, 1]
    z = z1 ** 2 - 2 * rho.squeeze() * z1 * z2 + z2 ** 2
    norm_const = 2 * np.pi * sigma[:, 0] * sigma[:, 1] * torch.sqrt(1 - rho.squeeze() ** 2)
    log_likelihood = -0.5 * z / (1 - rho.squeeze() ** 2) - torch.log(norm_const)
    return -torch.mean(log_likelihood)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Step 5: Train the Model
n_epochs = 20
model.train()
for epoch in range(n_epochs):
    epoch_loss = 0
    for inputs, targets in dataloader:
        inputs = inputs  # Add feature dimension
        targets = targets

        optimizer.zero_grad()
        mu, sigma, rho = model(inputs)
        loss = correlated_normal_loss(mu, sigma, rho, targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{n_epochs}, Loss: {epoch_loss / len(dataloader):.4f}")

# Step 6: Visualize Results Using Bokeh
model.eval()
preds_mu = []
preds_sigma = []
preds_rho = []
actual = []
with torch.no_grad():
    for i in range(len(returns) - seq_len):
        seq = returns[i:i + seq_len]
        mu, sigma, rho = model(seq)
        preds_mu.append(mu.numpy())
        preds_sigma.append(sigma.numpy())
        preds_rho.append(rho.item())
        actual.append(returns[i + seq_len].numpy())

# Convert to Pandas DataFrame
predictions = pd.DataFrame({
    'GBPUSD_Actual': [a[0] for a in actual],
    'EURUSD_Actual': [a[1] for a in actual],
    'GBPUSD_Mean': [m[0] for m in preds_mu],
    'EURUSD_Mean': [m[1] for m in preds_mu],
    'GBPUSD_Std': [s[0] for s in preds_sigma],
    'EURUSD_Std': [s[1] for s in preds_sigma],
    'Correlation': preds_rho
}, index=log_returns.index[seq_len:])

# Calculate 5th and 95th percentiles for both series
for currency in ['GBPUSD', 'EURUSD']:
    predictions[f'{currency}_Lower_Band'] = predictions[f'{currency}_Mean'] - 1.645 * predictions[f'{currency}_Std']
    predictions[f'{currency}_Upper_Band'] = predictions[f'{currency}_Mean'] + 1.645 * predictions[f'{currency}_Std']
    predictions[f'{currency}_Exceeds_Band'] = (
        (predictions[f'{currency}_Actual'] < predictions[f'{currency}_Lower_Band']) |
        (predictions[f'{currency}_Actual'] > predictions[f'{currency}_Upper_Band'])
    )

# Count exceedances
gbp_exceedances_count = predictions['GBPUSD_Exceeds_Band'].sum()
eur_exceedances_count = predictions['EURUSD_Exceeds_Band'].sum()

# Bokeh Visualization
output_notebook()
p = figure(title="GBP/USD and EUR/USD Weekly Log Returns: Actual vs. Predicted", x_axis_type="datetime", width=800, height=400)

# GBP/USD visualization
p.line(predictions.index, predictions['GBPUSD_Actual'], color='blue', legend_label='GBP/USD Actual', line_width=2)
p.line(predictions.index, predictions['GBPUSD_Mean'], color='green', legend_label='GBP/USD Predicted Mean', line_width=2)
p.varea(x=predictions.index, y1=predictions['GBPUSD_Lower_Band'], y2=predictions['GBPUSD_Upper_Band'], 
        color='green', alpha=0.2, legend_label='GBP/USD Uncertainty Band')

# EUR/USD visualization
p.line(predictions.index, predictions['EURUSD_Actual'], color='purple', legend_label='EUR/USD Actual', line_width=2)
p.line(predictions.index, predictions['EURUSD_Mean'], color='orange', legend_label='EUR/USD Predicted Mean', line_width=2)
p.varea(x=predictions.index, y1=predictions['EURUSD_Lower_Band'], y2=predictions['EURUSD_Upper_Band'], 
        color='orange', alpha=0.2, legend_label='EUR/USD Uncertainty Band')

# Add legend and labels
#p.legend.location = "top_left"
p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "Log Return"

# Display exceedance counts
print(f"GBP/USD Number of exceedances: {gbp_exceedances_count}")
print(f"EUR/USD Number of exceedances: {eur_exceedances_count}")

show(p)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Epoch 1/20, Loss: 0.0694
Epoch 2/20, Loss: -4.4316
Epoch 3/20, Loss: -5.4273
Epoch 4/20, Loss: -5.5029
Epoch 5/20, Loss: -5.7321
Epoch 6/20, Loss: -5.6407
Epoch 7/20, Loss: -5.7100
Epoch 8/20, Loss: -5.7522
Epoch 9/20, Loss: -5.7386
Epoch 10/20, Loss: -5.6714
Epoch 11/20, Loss: -5.7809
Epoch 12/20, Loss: -5.8633
Epoch 13/20, Loss: -5.7974
Epoch 14/20, Loss: -5.7760
Epoch 15/20, Loss: -5.7189
Epoch 16/20, Loss: -5.7969
Epoch 17/20, Loss: -5.7843
Epoch 18/20, Loss: -5.7704
Epoch 19/20, Loss: -5.6974
Epoch 20/20, Loss: -5.7611


Loading BokehJS ...

GBP/USD Number of exceedances: 127
EUR/USD Number of exceedances: 43


In [54]:
predictions.Correlation

Date
2004-02-18    0.432166
2004-02-25    0.432163
2004-03-03    0.432139
2004-03-10    0.432126
2004-03-17    0.432112
                ...   
2022-12-07    0.432109
2022-12-14    0.432187
2022-12-21    0.432129
2022-12-28    0.432178
2023-01-04    0.432151
Name: Correlation, Length: 984, dtype: float64

In [40]:
np.corrcoef(predictions.GBPUSD_Actual[:10], predictions.EURUSD_Actual[:10])

array([[1.        , 0.71037051],
       [0.71037051, 1.        ]])

In [38]:
df = pd.DataFrame([predictions.GBPUSD_Actual[:10], predictions.EURUSD_Actual[:10]])

In [36]:
predictions.GBPUSD_Actual.rolling(10).corr(predictions.EURUSD_Actual).dropna()

Date
2004-04-21    0.710371
2004-04-28    0.735734
2004-05-05    0.771065
2004-05-12    0.701889
2004-05-19    0.790936
                ...   
2022-12-07    0.916974
2022-12-14    0.887129
2022-12-21    0.720570
2022-12-28    0.727413
2023-01-04    0.726642
Length: 975, dtype: float64

In [56]:
predictions.EURUSD_Mean

Date
2004-02-18    0.002204
2004-02-25    0.002217
2004-03-03    0.002221
2004-03-10    0.002231
2004-03-17    0.002219
                ...   
2022-12-07    0.002184
2022-12-14    0.002215
2022-12-21    0.002180
2022-12-28    0.002214
2023-01-04    0.002208
Name: EURUSD_Mean, Length: 984, dtype: float32

In [60]:
predictions.EURUSD_Actual.rolling(10).mean().dropna()

Date
2004-04-21   -0.008284
2004-04-28   -0.007184
2004-05-05   -0.002743
2004-05-12   -0.002373
2004-05-19   -0.001882
                ...   
2022-12-07    0.008705
2022-12-14    0.006235
2022-12-21    0.009125
2022-12-28    0.007547
2023-01-04    0.006762
Name: EURUSD_Actual, Length: 975, dtype: float64